In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime

In [2]:
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False 
opciones.add_argument('--start-maximized')
# opciones.add_argument('--incognito')     

In [4]:
data = pd.read_csv("../mydata/cleandata/data.csv")
data.index=data.date
data.drop('date',axis=1,inplace=True)
foods = sorted(list(set(data['product'])))

In [11]:
driver = "../driver/chromedriver.exe"
driver = webdriver.Chrome(driver, options=opciones)
supermarket = "dia"
prices = []
prod = []
sup = []
links = []
names = []
newprice = []
frame = {}
for f in foods:
    url = f"https://www.dia.es/compra-online/search?text={f}&x=0&y=0"
    driver.get(url)
    time.sleep(1)
    try: #cookies
        driver.find_element_by_css_selector('#onetrust-accept-btn-handler').click()
    except:
        pass
    time.sleep(.5)
    for i in range(1,15):
        try:
            prices.append(driver.find_element_by_css_selector(f'#productgridcontainer > div.product-list--row > div:nth-child({i}) > div > a > div.column-right > div > p.pricePerKilogram').text.split("€")[0].strip())
            names.append(driver.find_element_by_css_selector(f'#productgridcontainer > div.product-list--row > div:nth-child({i}) > div > a > div.column-right > span').text)
            prod.append(f)
            sup.append(supermarket)
            links.append(url)
        except:
            pass
        
frame['price'] = prices
frame['product'] = prod
frame['supermarket'] = sup
frame['link'] = links
frame['name'] = names

scrap = pd.DataFrame(frame)
for i, row in scrap.iterrows():
    if row['price'] == '':
        newprice.append(np.nan)
    else:
        newprice.append(row['price'])
scrap['price'] = newprice
scrap.dropna(how='any',inplace=True)
scrap['price'] = [float((i.replace(",",".").replace("€","").strip())) for i in scrap['price']]
scrap['name'] = [(i.replace("%","por ciento").strip()) for i in scrap['name']]

if not os.path.exists(f"../mydata/scraps_{supermarket}"):
    os.makedirs(f"../mydata/scraps_{supermarket}")
now = str(datetime.now())[:19].replace(":","_")
scrap.to_csv(f'../mydata/scraps_{supermarket}/{now}.csv', index=False)
scrap.to_csv(f'../mydata/scraps_{supermarket}/{supermarket}.csv', index=False)

/tmp/ipykernel_715/2768875632.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver, options=opciones)
/tmp/ipykernel_715/2768875632.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#onetrust-accept-btn-handler').click()
/tmp/ipykernel_715/2768875632.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  prices.append(driver.find_element_by_css_selector(f'#productgridcontainer > div.product-list--row > div:nth-child({i}) > div > a > div.column-right > div > p.pricePerKilogram').text.split("€")[0].strip())
/tmp/ipykernel_715/2768875632.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  names.append(driver.find_element_by_css_selector(f'#productgridcontainer > div.product-list--row > div:nth-child({i}) > div > a > di